In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.ensemble import VotingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, RegressorMixin
import requests

# Define custom LSTM regressor
class LSTMRegressor(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = None

    def fit(self, X, y):
        self.model = Sequential()
        self.model.add(LSTM(50, activation='relu', input_shape=(X.shape[1], 1)))
        self.model.add(Dense(1))
        self.model.compile(optimizer='adam', loss='mse')
        self.model.fit(X, y, verbose=0)

    def predict(self, X):
        return self.model.predict(X).flatten()

# Coingecko API parameters
cryptocurrency = 'bitcoin'
currency = 'usd'
url = f'https://api.coingecko.com/api/v3/coins/markets?vs_currency={currency}&ids={cryptocurrency}'

# Make API request
response = requests.get(url)
data = response.json()

if len(data) > 0:
    open_price = data[0]['current_price']
    high_price = data[0]['high_24h']
    low_price = data[0]['low_24h']

    # Load dataset from CSV
    dataset = pd.read_csv('BTC.csv')
    X = dataset[['open_price', 'close_price', 'high_price', 'low_price']].values
    y = dataset['close_price'].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

    lstm_regressor = LSTMRegressor()
    tree_model = DecisionTreeRegressor()
    ensemble_model = VotingRegressor([('lstm', lstm_regressor), ('tree', tree_model)])

    lstm_regressor.fit(X_scaled[:, :2].reshape(X_scaled.shape[0], 2, 1), y_scaled)  # Use only first two columns for LSTM
    tree_model.fit(X_scaled[:, :2], y_scaled)  # Use only first two columns for DecisionTreeRegressor

    # Fit the ensemble model
    ensemble_model.fit(X_scaled[:, :2], y_scaled)  # Use only first two columns for ensemble model

    # Prepare input for prediction
    new_data = np.array([[open_price, 0, high_price, low_price]])  # Replace 0 with a suitable value for the previous day's closing price
    new_data_scaled = np.zeros_like(new_data)
    for i in range(new_data.shape[1]):
        new_data_scaled[:, i] = scaler.fit_transform(new_data[:, i].reshape(-1, 1)).flatten()

    new_data_scaled_reshaped = new_data_scaled[:, :2]  # Use only first two columns for prediction

    # Predict closing price using the ensemble model
    ensemble_prediction = ensemble_model.predict(new_data_scaled_reshaped.reshape(1, -1))
    ensemble_prediction = scaler.inverse_transform(ensemble_prediction.reshape(-1, 1)).flatten()
    print("Predicted closing price:", ensemble_prediction)
else:
    print("No data available for the specified cryptocurrency and currency.")


1/1 [==============================] - 0s 173ms/step
Predicted closing price: [26906.99893257]
